In [ ]:
from robobrowser import RoboBrowser
import geocoder
import folium

browser = RoboBrowser(history=True, parser='html5lib')
url = 'http://www.emol.com/noticias/nacional/2014/12/28/696553/ranking-los-100-colegios-con-mejor-puntaje-promedio-en-la-psu.html'
browser.open(url)


In [ ]:
rows = browser.select('#ranking-colegios-psu tbody tr')
colegios = []
for row in rows:
    cols = row.findAll('td')
    nombre = cols[1].text.strip()
    comuna = cols[2].text.strip()
    puntaje = float(cols[5].text.strip().replace(',','.'))
    while '  ' in nombre:
        nombre = nombre.replace('  ',' ')
    colegios.append((nombre, comuna, puntaje))
    


In [ ]:
posiciones = []
for nombre, comuna, puntaje in colegios:
    g=geocoder.google(nombre +", "+ comuna + ", chile")
    print g.latlng
    if g.latlng:
        posiciones.append((g.latlng, puntaje, nombre))

In [ ]:
avg_lat = sum([i[0] for i,j,k in posiciones])/len(posiciones)
avg_lon = sum([i[1] for i,j,k in posiciones])/len(posiciones)
min_puntaje = min([j for i,j,k in posiciones])
max_puntaje = max([j for i,j,k in posiciones])
print avg_lat, avg_lon

In [ ]:
from colour import Color
green = Color("green")
red = Color("red")
colors=list(green.range_to(red, 100))

In [ ]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=10)
i=0
posiciones.reverse()
for latlng, puntaje, nombre in posiciones:
    lat,lon = latlng
    radio = 10+(puntaje-min_puntaje)*(max_puntaje-min_puntaje)*20
    map.add_children(folium.CircleMarker([lat,lon],color='#000000',
                    fill_color=colors[i], radius=1000, popup=u"%s %s" % (nombre, puntaje)))
    i+=1
map.save('colegios.html')
map